# Setup

In [6]:
# Weights and Biases
#!pip install -q wandb
# Tensorflow
#!pip install -q tensorflow

In [1]:
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, Concatenate, Dense, BatchNormalization, LeakyReLU
from keras.activations import tanh
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import wandb
from wandb.keras import WandbCallback

In [2]:
import setup
#Erlend
wandb.login(key=setup.erlend_key)
# Hjalmar
#wandb.login(key=setup.hjalmar_key)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: erlendrygg (avogadro). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Erlend/.netrc


True

# Load, split and normalize data

### Load data

In [3]:
file = "../data/processed_data/2022.csv"
df_read = pd.read_csv(file)
display(df_read.head(30))

,Unnamed: 0,Quote_date,Expire_date,Price,Underlying_last,Strike,TTM,Underlying_1,Underlying_2,Underlying_3,...,Underlying_12,Underlying_13,Underlying_14,Underlying_15,Underlying_16,Underlying_17,Underlying_18,Underlying_19,Underlying_20,R
0,173719,2022-01-31,2022-02-02,3312.50,4516.89,1200.0,2,4431.8,4325.89,4347.26,...,4660.64,4726.55,4713.53,4669.85,4676.41,4696.25,4700.64,4793.19,4795.57,0.03
1,173720,2022-01-31,2022-02-02,3114.00,4516.89,1400.0,2,4431.8,4325.89,4347.26,...,4660.64,4726.55,4713.53,4669.85,4676.41,4696.25,4700.64,4793.19,4795.57,0.03
2,173721,2022-01-31,2022-02-02,2913.55,4516.89,1600.0,2,4431.8,4325.89,4347.26,...,4660.64,4726.55,4713.53,4669.85,4676.41,4696.25,4700.64,4793.19,4795.57,0.03
3,173722,2022-01-31,2022-02-02,2712.60,4516.89,1800.0,2,4431.8,4325.89,4347.26,...,4660.64,4726.55,4713.53,4669.85,4676.41,4696.25,4700.64,4793.19,4795.57,0.03
4,173723,2022-01-31,2022-02-02,2512.60,4516.89,2000.0,2,4431.8,4325.89,4347.26,...,4660.64,4726.55,4713.53,4669.85,4676.41,4696.25,4700.64,4793.19,4795.57,0.03
5,173724,2022-01-31,2022-02-02,2312.60,4516.89,2200.0,2,4431.8,4325.89,4347.26,...,4660.64,4726.55,4713.53,4669.85,4676.41,4696.25,4700.64,4793.19,4795.57,0.03
6,173725,2022-01-31,2022-02-02,2112.60,4516.89,2400.0,2,4431.8,4325.89,4347.26,...,4660.64,4726.55,4713.53,4669.85,4676.41,4696.25,4700.64,4793.19,4795.57,0.03
7,173726,2022-01-31,2022-02-02,1912.60,4516.89,2600.0,2,4431.8,4325.89,4347.26,...,4660.64,4726.55,4713.53,4669.85,4676.41,4696.25,4700.64,4793.19,4795.57,0.03
8,173727,2022-01-31,2022-02-02,1712.60,4516.89,2800.0,2,4431.8,4325.89,4347.26,...,4660.64,4726.55,4713.53,4669.85,4676.41,4696.25,4700.64,4793.19,4795.57,0.03
9,173728,2022-01-31,2022-02-02,1612.60,4516.89,2900.0,2,4431.8,4325.89,4347.26,...,4660.64,4726.55,4713.53,4669.85,4676.41,4696.25,4700.64,4793.19,4795.57,0.03


### Format input data

In [8]:
df = df_read

# Formating settings
val_start = "2022-11-01"
num_features = 4
time_steps = 20
bs_vars = ['Underlying_last', 'Strike', 'TTM', 'R']
underlying_lags = ['Underlying_last'] + [f'Underlying_{i}' for i in range (1, time_steps)]

# Split train and validation data
df_train = df[df['Quote_date'] < val_start]
df_val = df[df['Quote_date'] >= val_start]

# Extract target values
train_y = df_train['Price'].to_numpy()
val_y = df_val['Price'].to_numpy()

# Convert dataframes to numpy arrays
train_x = [df_train[underlying_lags].to_numpy(), df_train[bs_vars].to_numpy()]
val_x = [df_val[underlying_lags].to_numpy(), df_val[bs_vars].to_numpy()]

# Scale features based on training set
underlying_scaler = MinMaxScaler()
train_x[0] = underlying_scaler.fit_transform(train_x[0])
val_x[0] = underlying_scaler.transform(val_x[0])

bs_scaler = MinMaxScaler()
train_x[1] = bs_scaler.fit_transform(train_x[1])
val_x[1] = bs_scaler.transform(val_x[1])

# Shuffle training set
np.random.seed(0)
shuffle = np.random.permutation(len(train_x[0]))
train_x = [train_x[0][shuffle], train_x[1][shuffle]]
train_y = train_y[shuffle]

# Reshape data to fit LSTM
train_x = [train_x[0].reshape(len(train_x[0]),time_steps,1), train_x[1]]
val_x = [val_x[0].reshape(len(val_x[0]), time_steps, 1), val_x[1]]

print(f'Train shape: {train_x[0].shape}, {train_x[1].shape}')
print(f'Val shape: {val_x[0].shape}, {val_x[1].shape}')

Train shape: (1452255, 20, 1), (1452255, 4)
Val shape: (311065, 20, 1), (311065, 4)


# Model construction

In [5]:
def create_model(config):
    '''Builds an LSTM-MLP model of minimum 2 layers sequentially from a given config dictionary'''

    # Input layers
    underlying_history = Input((config.LSTM_timesteps,1))
    bs_vars = Input((config.Num_features,))

    # LSTM layers
    model = Sequential()

    model.add(LSTM(
        units = config.LSTM_units,
        activation = tanh,
        input_shape = (config.LSTM_timesteps, 1),
        return_sequences = True
    ))

    for _ in range(config.LSTM_layers - 2):
        model.add(LSTM(
            units = config.LSTM_units,
            activation = tanh,
            return_sequences = True
        ))
    
    model.add(LSTM(
        units = config.LSTM_units,
        activation = tanh,
        return_sequences = False
    ))

    # MLP layers
    layers = Concatenate()([model(underlying_history), bs_vars])
    
    for _ in range(config.MLP_layers - 1):
        layers = Dense(config.MLP_units)(layers)
        layers = BatchNormalization(momentum=config.Bn_momentum)(layers)
        layers = LeakyReLU()(layers)

    output = Dense(1, activation='relu')(layers)

    # Exponential decaying learning rate
    lr_schedule = ExponentialDecay(
        initial_learning_rate = config.Lr,
        decay_steps = int(len(train_x[0])/config.Minibatch_size),
        decay_rate=config.Lr_decay
    )

    # Compile model
    model = Model(inputs=[underlying_history, bs_vars], outputs=output)
    model.compile(loss='mse', optimizer=Adam(learning_rate=lr_schedule))

    model.summary()
    return model

# Hyperparameter search setup

In [ ]:
# Configuring the sweep hyperparameter search space
sweep_configuration = {
    'method': 'random',
    'name': 'LSTM-MLP v.1.0 testing',
    'metric': {
        'goal': 'minimize', 
        'name': 'val_loss'
		},
    'parameters': {
        'LSTM_units': {
            'values': [8, 16, 32, 64, 96, 128]},
        'MLP_units': {
            'values': [32, 64, 96, 128]},
        'LSTM_timesteps': {
            'values': [10, 20]},
        'LSTM_layers': {
            'distribution': 'int_uniform',
            'max': 6, 'min': 2},
        'MLP_layers': {
            'distribution': 'int_uniform',
            'max': 6, 'min': 2},
        'Bn_momentum': {
            'distribution': 'uniform',
            'max': 1, 'min': 0},
        'Lr': {
            'distribution': 'uniform',
            'max': 0.01, 'min': 0.0001},
        'Lr_decay': {
            'distribution': 'uniform',
            'max': 1, 'min': 0.9},        
        'Minibatch_size': {
            'values': [1024, 2048, 4096]},
        'Min_delta': {
            'value': 1},
        'Patience': {
            'value': 20},
        'Num_features': {
            'value': 4},
    }
}

# Initialize sweep and creating sweepID

# If new sweep, uncomment the line below and comment the line after it
#sweep_id = wandb.sweep(sweep=sweep_configuration, project='Deep learning for option pricing - test area') 
#sweep_id = '98bxt6oq'

Create sweep with ID: bu76nc4s
Sweep URL: https://wandb.ai/avogadro/Deep%20learning%20for%20option%20pricing%20-%20test%20area/sweeps/bu76nc4s


# Run hyperparameter search

In [6]:
def trainer(train_x = train_x, train_y = train_y, val_x = val_x, val_y = val_y, config = None, project = None):
    # Initialize a new wandb run
    with wandb.init(config=config, project = project):

        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        # Build model and create callbacks
        model = create_model(config)

        early_stopping = EarlyStopping(
            monitor='val_loss',
            mode='min',
            min_delta = config.Min_delta,
            patience = config.Patience,
        )
        
        wandb_callback = WandbCallback(
            monitor='val_loss',
            mode='min',
            save_model=False
        )

        # Adapt sequence length to config
        train_x[0] = train_x[0][:, :config.LSTM_timesteps, :]
        val_x[0] = val_x[0][:, :config.LSTM_timesteps, :]
        print(f'Train shape: {train_x[0].shape}, {train_x[1].shape}')
        print(f'Val shape: {val_x[0].shape}, {val_x[1].shape}')

        # Train model
        model.fit(
            train_x,
            train_y,
            batch_size = config.Minibatch_size,
            validation_data = (val_x, val_y),
            epochs = 1000,
            callbacks = [early_stopping, wandb_callback] 
        )

### Run full sweep

In [15]:
wandb.agent(sweep_id=sweep_id, function=trainer, project='Deep learning for option pricing - test area', count = 1)

wandb: Agent Starting Run: h2mmkvf0 with config:
wandb: 	Bn_momentum: 0.8268163490936014
wandb: 	LSTM_layers: 6
wandb: 	LSTM_timesteps: 10
wandb: 	LSTM_units: 8
wandb: 	Lr: 0.002322679981694606
wandb: 	Lr_decay: 0.9381204396211013
wandb: 	MLP_layers: 6
wandb: 	MLP_units: 32
wandb: 	Min_delta: 1
wandb: 	Minibatch_size: 2048
wandb: 	Num_features: 4
wandb: 	Patience: 20
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


2023-02-04 22:43:48.609942: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 10, 1)]      0           []                               
                                                                                                  
 sequential (Sequential)        (None, 8)            3040        ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 4)]          0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 12)           0           ['sequential[0][0]',             
                                                                  'input_2[0][0]']            

wandb: Ctrl + C detected. Stopping sweep.


### Single run

In [7]:
config = {
    'LSTM_units': 8,
    'MLP_units': 100,
    'LSTM_timesteps': 20,
    'LSTM_layers': 3,
    'MLP_layers': 4,
    'Bn_momentum': 0.99,
    'Lr': 0.01,
    'Lr_decay': 1,
    'Minibatch_size': 4096,
    'Min_delta': 1,
    'Patience': 20,
    'Num_features': 4,
    'Architecture': 'LSTM-MLP v.0.1',
    'Dataset': '2022: Val split 01.11 with shuffle',
}
trainer(config = config, project = 'Deep learning for option pricing - test area')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 20, 1)]      0           []                               
                                                                                                  
 sequential (Sequential)        (None, 8)            1408        ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 4)]          0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 12)           0           ['sequential[0][0]',             
                                                                  'input_2[0][0]']            

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▂▁▂▁▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,81
best_val_loss,293.61331
epoch,101
loss,386.87576
val_loss,3117.26147
